# 데이터 증식을 이용하여 이미지 분류하기

## 모델 생성의 두번 째 방법

In [3]:
### 필요한 라이브러리 임폴트
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import random

In [4]:
'''
모델 생성의 첫번 쨰 방법
model = tf.keras.Sequental()
model.add(tf.keras.layer.Conv2d()
'''

'''
### 모델 생성의 두번 째 방법
model = tf.keras.Model(input, output)
'''
# 인풋에 널고 아웃풋 결과만 널어 모델을 만드는 구조만 만들어두면 뚝딱 만들어짐
# 배열 만들고 어떤 함수에 입력으로 주고 그 결과 값을 새로운 함수 입력에 넣으면 또 결과값이 나오는데 이걸 반복
# 딥러닝 컵퓨터 비전 p.48 맨위에것 써서 p.51 모델을 만들음 (강아지그림)

# 입력 생성용 전용 합수
inputs = tf.keras.Input(shape=(3,))

# 출력 생성
outputs = tf.keras.layers.Dense(units=4, activation='softmax')(inputs)

# 모델 생성
model = tf.keras.Model(inputs, outputs)

# 결과 확인
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense (Dense)               (None, 4)                 16        
                                                                 
Total params: 16 (64.00 Byte)
Trainable params: 16 (64.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## 모델 생성

In [5]:
### 기본 모델 생성하기
base_model = tf.keras.applications.vgg16.VGG16(weights=None)

# 모델 구조 확인
base_model.summary()

# TensorFlow 에서 제공하는 여러 모델들 중 vgg16이라는 모델에서 vgg16이라는 파이썬 파일이 있음 (okk.OKK가 있듯)
# Keras / keras / applications / vgg16
# 그냥 불러오면 있는 것들 다 가져오니까 weight (지정할게 없거나, 사전학습된 모델을 넣고 싶으면 가중치를 다 가지고 있는 모델을 가져옴)


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0    

In [6]:
### 학습 전 기본 모델의 가중치 확인
print(base_model.weights)

# weight에 가중치 안받겠다고 해서 다 0 나오는거

[<tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 3, 64) dtype=float32, numpy=
array([[[[ 0.09501095, -0.00536808, -0.0282796 , ...,  0.02954663,
          -0.06910515, -0.04808179],
         [ 0.05973376,  0.04778557,  0.09295805, ...,  0.07414639,
          -0.03237057,  0.06521355],
         [ 0.02176882, -0.09317692, -0.0143269 , ...,  0.08030669,
           0.05322342, -0.06236446]],

        [[ 0.08148195,  0.04973444, -0.04225602, ..., -0.05951667,
           0.0049757 , -0.08117792],
         [ 0.0206093 ,  0.07843313, -0.09156472, ..., -0.0481021 ,
          -0.05517789,  0.04651444],
         [-0.09580889,  0.05632527, -0.03587782, ...,  0.02112681,
           0.03573902, -0.08774847]],

        [[-0.02703758,  0.04785125, -0.06318238, ...,  0.07204141,
           0.05163465, -0.06719656],
         [ 0.0433951 ,  0.06208551,  0.00777839, ..., -0.0539256 ,
          -0.09603597,  0.06937291],
         [-0.0982496 , -0.06939332,  0.06983063, ...,  0.00757684,
          -0.0873

In [7]:
### 기본 모델의 각 계층(layer) 별 가중치 확인하기

# 생성된 모델의 전체 계층(layer) 확인
layers = base_model.layers
print(f'생성된 모델의 전체 계층 확인 : \n{layers}')

print('-'*88)

# 계층별 가중치 확인
for layer in layers:
    print(f'name = {layer.name}')
    print(f'weights : \n{layer.weights}')
    print('='*88)

생성된 모델의 전체 계층 확인 : 
[<keras.src.engine.input_layer.InputLayer object at 0x0000028AB65A4F50>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x0000028AB3946A10>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x0000028AB6940290>, <keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x0000028AB6941090>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x0000028AB66A6C10>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x0000028AB67DE750>, <keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x0000028AB68B3250>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x0000028AB6927E50>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x0000028AB5FFD510>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x0000028AB68B2E90>, <keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x0000028AB6925A10>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x0000028AB6A02110>, <keras.src.layers.convolutional.conv2d.

weights : 
[<tf.Variable 'block3_conv3/kernel:0' shape=(3, 3, 256, 256) dtype=float32, numpy=
array([[[[-8.56102444e-03, -3.22585888e-02, -1.10288765e-02, ...,
          -1.77969448e-02, -3.22399288e-02, -1.37785673e-02],
         [-9.19137150e-03,  1.59297436e-02,  8.40646029e-03, ...,
          -1.25379264e-02, -4.41491604e-03, -1.51463971e-02],
         [ 8.46301019e-03, -2.39363946e-02,  2.94670537e-02, ...,
           1.61737651e-02, -3.26826796e-02,  2.70364583e-02],
         ...,
         [-2.78276354e-02,  5.94633445e-03,  4.05099243e-03, ...,
           1.44566521e-02,  1.25642195e-02,  1.58894695e-02],
         [ 3.53835151e-02, -7.19692931e-03,  2.36410908e-02, ...,
           6.38841838e-03, -2.80053597e-02,  2.44568363e-02],
         [-4.90096956e-03,  8.90474766e-04, -2.76676416e-02, ...,
          -4.28804383e-03, -2.66565606e-02, -4.93004918e-04]],

        [[ 6.10068440e-04,  2.39739753e-02,  2.65629664e-02, ...,
          -5.23103215e-03,  1.94384158e-02,  1.73775144e

In [8]:
### 기본 모델의 각 계층 인덱스와 이름 확인 (가중치는 너무 기니까 패스)

 # enumerate() 함수 --> index 생성  / (enumerate = 인덱스 자동 생성해주는 기능)
for idx,layer in enumerate(base_model.layers):
    print(f'인덱스 = {idx}, 이름 = {layer.name}')   # name 자리에 weights를 넣으면 가중치 확인 가능

인덱스 = 0, 이름 = input_2
인덱스 = 1, 이름 = block1_conv1
인덱스 = 2, 이름 = block1_conv2
인덱스 = 3, 이름 = block1_pool
인덱스 = 4, 이름 = block2_conv1
인덱스 = 5, 이름 = block2_conv2
인덱스 = 6, 이름 = block2_pool
인덱스 = 7, 이름 = block3_conv1
인덱스 = 8, 이름 = block3_conv2
인덱스 = 9, 이름 = block3_conv3
인덱스 = 10, 이름 = block3_pool
인덱스 = 11, 이름 = block4_conv1
인덱스 = 12, 이름 = block4_conv2
인덱스 = 13, 이름 = block4_conv3
인덱스 = 14, 이름 = block4_pool
인덱스 = 15, 이름 = block5_conv1
인덱스 = 16, 이름 = block5_conv2
인덱스 = 17, 이름 = block5_conv3
인덱스 = 18, 이름 = block5_pool
인덱스 = 19, 이름 = flatten
인덱스 = 20, 이름 = fc1
인덱스 = 21, 이름 = fc2
인덱스 = 22, 이름 = predictions


In [9]:
### 출력층 수정 --> 재구성 모델 생성

'''
### 기존 모델의 출력 units(정답 레이블)의 수 = 1000개
### 우리에게 필요한 출력 units=2 --> 강아지 / 고양이
### 출력층 수정 --> 재구성 모델 생성
'''

# 출력 units 설정
units = 2

# 출력층 수정
x = base_model.get_layer(index=21).output
predictions = tf.keras.layers.Dense(units=units, activation='softmax')(x)#이거자체가 함수 x

###### f(x)

# 21번째의 출력이 있어야 22번째 재정의하는 곳에 데이터를 집어넣어서 데이터값을 끌어낼 수 있음.
# 

### 재구성 모델 생성

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# 결과 확인

print(f'재구성한 모델의 구조')
model.summary()



'''
tf.keras.Sequential()과

tf.keras.Model()는 매우 다르다 
차이점 복습하기 
'''



재구성한 모델의 구조
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128

'\ntf.keras.Sequential()과\n\ntf.keras.Model()는 매우 다르다 \n차이점 복습하기 \n'

In [10]:
base_model.input

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_2')>

In [11]:
base_model.get_layer(index=0).output

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_2')>

In [12]:
## 위에 출력층 둘다 같은 식.

## 데이터 증식

In [18]:
## zip 파일 압축 해제

!unzip 'D:\Code\DataSets/cats_and_dogs.zip' -d 'D:\Code\DataSets'


'unzip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [26]:
### 이미지의 수 확인
import os
# 이미지 폴더 경로 설정
train_dir = 'D:\Code\DataSets\cats_and_dogs/train'
val_dir = 'D:\Code\DataSets\cats_and_dogs/val'
test_dir = 'D:\Code\DataSets\cats_and_dogs/test'

# 학습용 폴더에 들어있는 강아지 이미지의 수 확인
dogs_list = os.listdir(train_dir + '/' + 'dogs')
# dogs_list = os.listdir(os.path.join(train_dir, 'dogs'))
print(f'학습용 dogs 폴더에 들어있는 강아지 이미지 파일의 이름 : \n{dogs_list}')

print('*'*80)

print(f'학습용 dogs 폴더에 들어있는 강아지 이미지의 수 = {len(dogs_list)}개')

# os는 이미지처리할 때 많이 쓰임.

print('*'*80)

# 검증용 폴더에 들어있는 고양이 이미지의 수 확인
cats_list = os.listdir(train_dir + '/' + 'cats')

print(f'학습용 cats 폴더에 들어있는 고양이 이미지 파일의 이름 : \n{cats_list}')

print('*'*80)

print(f'학습용 cats 폴더에 들어있는 고양이 이미지의 수 = {len(cats_list)}개')


print('*'*80)


학습용 dogs 폴더에 들어있는 강아지 이미지 파일의 이름 : 
['dog.0.jpg', 'dog.1.jpg', 'dog.10.jpg', 'dog.100.jpg', 'dog.101.jpg', 'dog.102.jpg', 'dog.103.jpg', 'dog.104.jpg', 'dog.105.jpg', 'dog.106.jpg', 'dog.107.jpg', 'dog.108.jpg', 'dog.109.jpg', 'dog.11.jpg', 'dog.110.jpg', 'dog.111.jpg', 'dog.112.jpg', 'dog.113.jpg', 'dog.114.jpg', 'dog.115.jpg', 'dog.116.jpg', 'dog.117.jpg', 'dog.118.jpg', 'dog.119.jpg', 'dog.12.jpg', 'dog.120.jpg', 'dog.121.jpg', 'dog.122.jpg', 'dog.123.jpg', 'dog.124.jpg', 'dog.125.jpg', 'dog.126.jpg', 'dog.127.jpg', 'dog.128.jpg', 'dog.129.jpg', 'dog.13.jpg', 'dog.130.jpg', 'dog.131.jpg', 'dog.132.jpg', 'dog.133.jpg', 'dog.134.jpg', 'dog.135.jpg', 'dog.136.jpg', 'dog.137.jpg', 'dog.138.jpg', 'dog.139.jpg', 'dog.14.jpg', 'dog.140.jpg', 'dog.141.jpg', 'dog.142.jpg', 'dog.143.jpg', 'dog.144.jpg', 'dog.145.jpg', 'dog.146.jpg', 'dog.147.jpg', 'dog.148.jpg', 'dog.149.jpg', 'dog.15.jpg', 'dog.150.jpg', 'dog.151.jpg', 'dog.152.jpg', 'dog.153.jpg', 'dog.154.jpg', 'dog.155.jpg', 'dog.156.jpg'

# 데이터 증식(augmentation)

In [ ]:
## 데이터 증식의 의미와 방법
#- 의미 : 원본 이미지에 인위적인 변화를 주어 학습 데이터의 양을 늘린다. 과대적합 방지
#- 증식의 방법
#1. 회전(rotation)
#2. 이동(shift)
#3. 밀림(shearing)
#4. 확대/축소(zoom)
#5. 상하좌우 반전(flip) 등

In [ ]:
## tf.keras에서 제공하는 데이터 증식과 관련된 유용한 함수들
#
#- ImageDataGenerator() : 하이퍼파라미터 설정을 통해서 데이터 증식 방식에 대해서 설정
#train_datagen = ImageDataGenerator(하이퍼파라미터)
#val_datagen = ImageDataGenerator ... 이후 pdf 참조